In [1]:
import requests
import tiktoken
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

## Getting the data

In [2]:
docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [7]:
query = "How do execute a command on a Kubernetes pod?"

In [8]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            }
        }
    },
}

response = es_client.search(index=index_name, body=search_query)

In [9]:
top_score = response["hits"]["hits"][0]["_score"]
print(f"Top-ranking result score: {top_score}")

Top-ranking result score: 44.50556


## Q4. Filtering

In [10]:
query = "How do copy a file to a Docker container?"

In [11]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
            "filter": {"term": {"course": "machine-learning-zoomcamp"}},
        }
    },
}

response = es_client.search(index=index_name, body=search_query)

In [12]:
print("The 3rd question returned by the search engine is:", response["hits"]["hits"][2]["_source"]["question"])

The 3rd question returned by the search engine is: How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = (
            context
            + f"Q: {doc['_source']['question']}\nA: {doc['_source']['text']}\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
prompt = build_prompt(query, response["hits"]["hits"])

In [15]:
print(f"The length of the resulting prompt is {len(prompt)}")

The length of the resulting prompt is 1447


## Q6. Tokens

In [16]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [17]:
encoded_prompt = encoding.encode(prompt)

In [18]:
print(f"The prompt has {len(encoded_prompt)} tokens.")

The prompt has 321 tokens.
